In [ ]:
! pip install transformers
! pip install datasets
! pip install evaluate
! pip install scipy sklearn
! pip install tensorflow

In [2]:
model_checkpoint = "bert-base-cased"
learning_rate = 1e-5
num_epochs = 1
batch_size = 14
max_length = 16
num_labels = 5
task = "stsb"

In [3]:
from datasets import load_dataset

In [4]:
dataset = load_dataset("yelp_review_full")

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset yelp_review_full downloaded and prepared to /root/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})


In [ ]:
from evaluate import load

#(Semantic Textual Similarity Benchmark) Determine the similarity of two sentences with a score from 1 to 5.
metric = load("glue", task)
# references = [0., 1., 2., 3., 4., 5.]
# predictions = [0., 1., 2., 3., 4., 5.]
print(metric)

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [8]:
def tokenize_data(data):
  return tokenizer(data["text"], truncation=True)

In [9]:
encoded_dataset = dataset.map(tokenize_data, batched=True)

  0%|          | 0/650 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [10]:
print(encoded_dataset)


DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})


In [11]:
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf
    
id2label = {1:'1 star', 2:'2 star', 3:'3 stars', 4:'4 stars', 5:'5 stars'}
label2id = {val: key for key, val in id2label.items()}

model = TFAutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id
)

Downloading:   0%|          | 0.00/527M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from datasets import Dataset
train_dataset = encoded_dataset.filter(lambda example, indice: indice < 100, with_indices=True)
test_dataset = encoded_dataset.filter(lambda example, indice: indice < 10, with_indices=True)


  0%|          | 0/650 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

  0%|          | 0/650 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [13]:
print(train_dataset)
print(test_dataset)

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 100
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 100
    })
})
DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 10
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 10
    })
})


In [14]:

tf_train_dataset = model.prepare_tf_dataset(
    train_dataset['train'],
    shuffle=True,
    batch_size=batch_size,
    tokenizer=tokenizer
)

tf_validation_dataset = model.prepare_tf_dataset(
    test_dataset['test'],
    shuffle=False,
    batch_size=batch_size,
    tokenizer=tokenizer
)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [15]:
print(tf_train_dataset.element_spec)
print(tf_validation_dataset.element_spec)

({'input_ids': TensorSpec(shape=(14, None), dtype=tf.int64, name=None), 'token_type_ids': TensorSpec(shape=(14, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(14, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(14,), dtype=tf.int64, name=None))
({'input_ids': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'token_type_ids': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))


In [16]:
from transformers import create_optimizer

batches_per_epoch = len(encoded_dataset["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)

optimizer, schedule = create_optimizer(
    init_lr=learning_rate, num_warmup_steps=0, num_train_steps=total_train_steps
)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [17]:
from transformers.keras_callbacks import KerasMetricCallback
import numpy as np


def compute_metrics(eval_predictions):
    predictions, labels = eval_predictions
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)


metric_callback = KerasMetricCallback(
    metric_fn=compute_metrics, eval_dataset=tf_validation_dataset
)

In [ ]:
# from transformers.keras_callbacks import PushToHubCallback
# from tensorflow.keras.callbacks import TensorBoard

# model_name = model_checkpoint.split("/")[-1]
# push_to_hub_model_id = f"{model_name}-finetuned-{task}"

# tensorboard_callback = TensorBoard(log_dir="./text_classification_model_save/logs")

# push_to_hub_callback = PushToHubCallback(
#     output_dir="./text_classification_model_save",
#     tokenizer=tokenizer,
#     hub_model_id=push_to_hub_model_id,
# )

# callbacks = [metric_callback]

model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=num_epochs
)